<a href="https://colab.research.google.com/github/joshuaghannan/ECEC247_Project/blob/jgh_tests/gridsearch_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### loading modules and data

In [1]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# # If using colab
# from google.colab import files
# files.upload()
# # select the 3 .py files (models, utils, data_utils)

In [3]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

EEG_loading.ipynb	PyTorch_Experiments.ipynb  X_train_valid.npy
person_test.npy		pytorch_rnn_example.ipynb  y_test.npy
person_train_valid.npy	X_test.npy		   y_train_valid.npy


In [4]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [5]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
# Uncomment for window
train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
# num_levels = 5
# top_scale = 0.4
# X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, top_scale=top_scale)
# train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
# X_cwt[train_fold[0]].shape

### Grid Search over window sizes
Creates a pd table for each stride size

In [0]:
import pandas as pd 
pd.set_option("display.precision", 2) # set the precision to 2 decimals

In [8]:
# setting up the data augmentation here
aug_type = "window"
strides = [20, 50, 100]
window_sizes = [100, 200, 300, 500, 750, 900]
vote_num = 20 # Make 20 normally
num_folds = 1
num_epochs = 20

print('starting grid search...\n')
for stride in strides:

  print('--------------------------------------------------')
  print('Stride: {}'.format(stride))
  # print('----------')

  test_accs = []
  for window_size in window_sizes:
    for k in range(num_folds):
        # indicate hyperparameters here
        model, criterion, optimizer = InitRNN(rnn_type='CNNGRUnet', dropout=0.6, weight_decay=0.00001)
        # print ('fold {}'.format(k+1))
        
        # Uncomment for just windowing
        X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
        X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

        # Uncomment for cwt
        # X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
        # X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]

        X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
        if aug_type != 'window':
            X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
        EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
        EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
        EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
        EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
        best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num, verbose=False)

    test_accs.append(100*TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num))
  
  df = pd.DataFrame(list(zip(window_sizes, test_accs)),  
                  columns = ['Window Size', 'Test Accuracy']) 
  display(df)
  print('\n')

starting grid search...

--------------------------------------------------
Stride: 20


,Window Size,Test Accuracy
0,100,33.41
1,200,32.28
2,300,33.18
3,500,23.25
4,750,29.57
5,900,21.90




--------------------------------------------------
Stride: 50


KeyboardInterrupt: ignored